<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/%E6%83%85%E5%A0%B1%E8%AA%8D%E8%AD%98%E8%AB%962021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 情報認識論試験問題

以下の問題について解答し、結果を提出してください。なお、解答については文章はテキストセルに、プログラムに関してはコードセルに記入すること。また、結果の提出については、この画面の左上の「ファイル」メニュー内にある「ダウンロード」->「.ipynbをダウンロード」で自分のマシンにダウンロードできるので、それを各自の学内メールに添付してください。

#1. ニューラルネット

(1) ニューラルネットを学習させる際に用いられる「誤差逆伝播法」について説明せよ。

【解答欄】

(2) 誤差逆伝播法では、微分操作が必要となる。以下のネットワークを考える。

- 入力層：$x = (x_1, x_2, ..., x_i, ..., x_n)$
- 第１層：$y = x + 2$
- 第２層：$z = y \odot y \times 3$ (注：$\odot$ はアダマール積（成分ごとの積）)
- 出力層：$out = mean(z)$ （$z_i$の平均値）

このとき、出力値に対する入力値の微分
$\frac{\partial}{\partial x_{i}}out$ を計算せよ。

【解答欄】
$$ \frac{\partial}{\partial x_{i}}out = ここに数式を書く $$

(3) $x = (1, 2, 3, 4)$ となった時の、$\frac{\partial}{\partial x}out$ を求めよ。





【解答欄】
$$
(\frac{\partial}{\partial x_1}out, \frac{\partial}{\partial x_2}out, \frac{\partial}{\partial x_3}out, \frac{\partial}{\partial x_4}out) = ここに数値を書く
$$

# 2. Pytorch

(1) 本講義では、ニューラルネットの実装に Pytorch を用いたが、そこで用いられる基本的なデータ構造に「テンソル」がある。テンソルとはどのようなもので、ニューラルネットにおいてはどのように使われるかを説明せよ。

【解答欄】

(2) 2 x 2 のリスト [[1, 2], [3, 4]] からテンソルを作り、x という変数に代入せよ。

（ヒント：テンソルの作成には、torch.Tensor() を用いる）

In [2]:
import torch

l = [[1, 2], [3, 4]]         # ここにリストを書く
print("list: ", l)

x = torch.Tensor(l)         # ここでテンソルに変換
print("x: ", x)

list:  [[1, 2], [3, 4]]
x:  tensor([[1., 2.],
        [3., 4.]])


Pytorch には微分を自動で行なってくれるため、上記の誤差逆伝播法が簡単に実行できる。そのためには、テンソルの requires_grad 属性を True にしておく必要がある。


In [3]:
x.requires_grad = True # これはそのまま実行してください。

(3) 問１でのネットワークをテンソル x を用いて構築せよ。

In [4]:
y = x + 2# ここを自分で埋める
z = y * y * 3# ここも自分で埋める埋める
out = z.mean() # ここも自分で埋める

(4) backword()関数を用いて、out に対する x の微分値を求めよ。

In [5]:
out.backward() # これで逆伝播を行う
x.grad # x の grad 属性に微分値が格納される

tensor([[4.5000, 6.0000],
        [7.5000, 9.0000]])

# 3. Pytorch を用いた実例

In [6]:
# ライブラリのインポート
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
 
# PyTorchのインポート
import torch 
import torch.nn as nn
 
# 評価指標（Scikit-learn）
from sklearn.metrics import mean_squared_error

データの取得（ラーメンデータベース）

In [8]:
!curl https://raw.githubusercontent.com/ailab-nda/NLP/main/ramen-ratings.csv > ramen-ratings.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  154k  100  154k    0     0   844k      0 --:--:-- --:--:-- --:--:--  844k


ダウンロードしたデータの読み込み

In [9]:
ramen = pd.read_csv('ramen-ratings.csv')

###データの確認
（最初の5行を表示）

In [10]:
ramen.head()

,Review #,Brand,Variety,Style,Country,Stars,Top Ten
0,2580,New Touch,T's Restaurant Tantanmen,Cup,Japan,3.75,NaN
1,2579,Just Way,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,Pack,Taiwan,1,NaN
2,2578,Nissin,Cup Noodles Chicken Vegetable,Cup,USA,2.25,NaN
3,2577,Wei Lih,GGE Ramen Snack Tomato Flavor,Pack,Taiwan,2.75,NaN
4,2576,Ching's Secret,Singapore Curry,Pack,India,3.75,NaN


データのサイズ確認

In [11]:
ramen.shape

(2580, 7)

ラーメンの国別登録数の確認

In [14]:
ramen['Country'].value_counts()[0:10]

Japan          352
USA            323
South Korea    309
Taiwan         224
Thailand       191
China          169
Malaysia       156
Hong Kong      137
Indonesia      126
Singapore      109
Name: Country, dtype: int64

未評価（Unrated）のラーメンを除外

In [15]:
mask = ramen.index[ramen['Stars'] == 'Unrated']
ramen = ramen.drop(index = mask)
ramen.shape

(2577, 7)

データの変換

In [16]:
# Starsのデータ型を確認
print(ramen['Stars'].dtype)
 
# float型へ変換
ramen['Stars'] = ramen['Stars'].astype(float)
 
# 改めてデータ型を確認
print(ramen['Stars'].dtype)

object
float64


不要なカラムをデータから削除

In [17]:
ramen = ramen.drop(columns=['Review #', 'Top Ten', 'Variety'])
ramen.head()

,Brand,Style,Country,Stars
0,New Touch,Cup,Japan,3.75
1,Just Way,Pack,Taiwan,1.00
2,Nissin,Cup,USA,2.25
3,Wei Lih,Pack,Taiwan,2.75
4,Ching's Secret,Pack,India,3.75


特徴量のダミー変数化

In [18]:
Country = pd.get_dummies(ramen['Country'], prefix='Country', drop_first=True)
Brand = pd.get_dummies(ramen['Brand'], prefix='Brand',drop_first=True)
Style = pd.get_dummies(ramen['Style'], prefix='Style',drop_first=True)
 
# ダミー変数化した特徴量を結合
ramendf = pd.concat([Country, Brand,Style], axis=1)
 
# 確認
ramendf.head()


,Country_Bangladesh,Country_Brazil,Country_Cambodia,Country_Canada,Country_China,Country_Colombia,Country_Dubai,Country_Estonia,Country_Fiji,Country_Finland,Country_Germany,Country_Ghana,Country_Holland,Country_Hong Kong,Country_Hungary,Country_India,Country_Indonesia,Country_Japan,Country_Malaysia,Country_Mexico,Country_Myanmar,Country_Nepal,Country_Netherlands,Country_Nigeria,Country_Pakistan,Country_Philippines,Country_Poland,Country_Sarawak,Country_Singapore,Country_South Korea,Country_Sweden,Country_Taiwan,Country_Thailand,Country_UK,Country_USA,Country_United States,Country_Vietnam,Brand_7 Select,Brand_7 Select/Nissin,Brand_A-One,...,Brand_Unif Tung-I,Brand_Unif-100,Brand_United,Brand_Unox,Brand_Unzen,Brand_Urban Noodle,Brand_Ve Wong,Brand_Vedan,Brand_Vifon,Brand_Vina Acecook,Brand_Vit's,Brand_Wai Wai,Brand_Wang,Brand_Weh Lih,Brand_Wei Chuan,Brand_Wei Lih,Brand_Wei Wei,Brand_Westbrae,Brand_Western Family,Brand_World O' Noodle,Brand_Wu Mu,Brand_Wu-Mu,Brand_Wugudaochang,Brand_Xiao Ban Mian,Brand_Xiuhe,Brand_Yamachan,Brand_Yamadai,Brand_Yamamori,Brand_Yamamoto,Brand_Yum Yum,Brand_Yum-Mie,Brand_Zow Zow,Brand_iMee,Brand_iNoodle,Style_Bowl,Style_Box,Style_Can,Style_Cup,Style_Pack,Style_Tray
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


データを特徴量とターゲットへ分割

In [19]:
X = np.array(ramendf, dtype=np.float32) 
y = np.array(ramen[['Stars']], dtype=np.float32)

## モデルの作成

In [20]:
# モデル
model = nn.Linear(397, 1)

# 損失関数
loss = nn.MSELoss()
 
# 最適化関数
optimizer = torch.optim.SGD(model.parameters(), lr=0.6)

モデルの学習

In [21]:
for epoch in range(1000):
    # ステージ1 Numpy配列からテンソルへ変換
    inputs = torch.from_numpy(X)
    targets = torch.from_numpy(y)

    # ステージ2 推測値を出力して誤差（コスト）を算出
    outputs = model(inputs)
    cost = loss(outputs, targets)
    # ステージ3 誤差逆伝播（バックプロパゲーション）
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # ステージ4 50回毎にコストを表示
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 1000, cost.item()))

Epoch [100/1000], Loss: 0.7753
Epoch [200/1000], Loss: 0.7268
Epoch [300/1000], Loss: 0.6980
Epoch [400/1000], Loss: 0.6781
Epoch [500/1000], Loss: 0.6634
Epoch [600/1000], Loss: 0.6519
Epoch [700/1000], Loss: 0.6426
Epoch [800/1000], Loss: 0.6349
Epoch [900/1000], Loss: 0.6284
Epoch [1000/1000], Loss: 0.6228


予測値の出力

In [23]:
y_pred = model(torch.from_numpy(X)).data.numpy()

#最初の５つを確認
print(y_pred[0:5])
print(y[0:5])

[[4.111941 ]
 [2.9364026]
 [3.4577096]
 [3.4201987]
 [3.6928964]]
[[3.75]
 [1.  ]
 [2.25]
 [2.75]
 [3.75]]


予測の平均二乗誤差を確認

In [24]:
mean_squared_error(y, y_pred)

0.62279326

問題：日清のでかぶと豚キムチのレーティングは何点と予測されるか？また、実際は何点か？

In [25]:
## ここで解答プログラムを実行